In [64]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [53]:
def preprocess(im,height,width):
    #if the image is already bigger in either height or width the image is cut to the correct size (this is a temperary solution)
    if im.shape[0] > height:
        im = im[:height,:,:]
    if im.shape[1] > width:
        im = im[:,:width,:]
    #image changed to float in the range 0-1
    im = im[:,:,:7]/255
    #create a simple mask, and make everything else 0
    mask = im[:,:,4].copy()
    mask[im[:,:,4] < 0.175] = 0
    mask[im[:,:,4] >= 0.175] = 1
    im[mask==0] = 0
    
    #Pad the image to the correct size
    height_diff = height-im.shape[0]
    width_diff = width-im.shape[1]
    if height_diff%2 != 0:
        top = int(height_diff/2)
        bottom = int(height_diff/2)+1
    else:
        top = int(height_diff/2)
        bottom = top
    if width_diff%2 != 0:
        left = int(width_diff/2)
        right = int(width_diff/2)+1
    else:
        left = int(width_diff/2)
        right = left
    im = cv2.copyMakeBorder(im,top,bottom,left,right,cv2.BORDER_CONSTANT,0)
    return np.moveaxis(im,-1,0)

In [54]:
# classes = os.listdir('../data')
# files = []
# for folder in classes:
#     files.append(os.listdir('../data/'+folder))
    
# for i in range(len(files)):
#     for j in range(len(files[i])):
#         files[i][j] = '../data/'+classes[i]+'/'+files[i][j]

# class_len = []
# for i in range(len(files)):
#     class_len.append(len(files[i]))
# labels = np.zeros((66000,5))
# for i in range(class_len[0]):
#     labels[i,0] = 1
# for i in range(class_len[0],sum(class_len[:2])):
#     labels[i,1] = 1
# for i in range(sum(class_len[:2]),sum(class_len[:3])):
#     labels[i,2] = 1
# for i in range(sum(class_len[:3]),sum(class_len[:4])):
#     labels[i,3] = 1
# for i in range(sum(class_len[:4]),sum(class_len[:5])):
#     labels[i,4] = 1
    
# files = np.array([val for sublist in files for val in sublist])
# np.save("data_path",files)
# np.save("labels",labels)
data=np.load("data_path.npy")
labels=np.load("labels.npy")

Class 0: Oat

Class 1: Broken

Class 2: Wheat

Class 3: Barley

Class 4: Rye

In [55]:
class dataset (Dataset):
    def __init__(self,data,target,height,width):
        self.data = data
        self.target = target
        self.height = height
        self.width = width
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        temp = np.zeros((len(idx),7,self.height,self.width))
        to_be_loaded = self.data[idx]
        for item,i in zip(to_be_loaded,range(len(idx))):
            temp[i,:,:,:] = preprocess(np.load(item),self.height,self.width)
        
        X = temp
        y = self.target[idx]
        
        return X,y